### **GPT-4o-Mini w/ Augmentation** ###

#### Configuration

In [1]:
# Dependencies
import os
from config import DARTMOUTH_API_KEY, DARTMOUTH_CHAT_API_KEY
from langchain_dartmouth.llms import ChatDartmouth

# Retrieving keys and creating environment variables
os.environ['DARTMOUTH_CHAT_API_KEY'] = DARTMOUTH_CHAT_API_KEY
os.environ['DARTMOUTH_API_KEY'] = DARTMOUTH_API_KEY

# Defining llm and embeddings models
llm_model_name = "openai.gpt-4o-mini-2024-07-18"
embeddings_model_name = "bge-m3"

# Defining keywords and sources
keywords = "social security OR government welfare OR medicare OR medicaid"
source="breitbart-news"

# Defining testing data file
testing_data = '../input/govtspending.json'

In [7]:
from langchain_dartmouth.llms import DartmouthLLM
print(DartmouthLLM.list())

[{'name': 'llama-3-8b-instruct', 'provider': 'meta', 'display_name': 'Llama 3 8B Instruct', 'tokenizer': 'meta-llama/Meta-Llama-3-8B-Instruct', 'type': 'llm', 'capabilities': ['chat'], 'server': 'text-generation-inference', 'parameters': {'max_input_tokens': 8192}}, {'name': 'llama-3-2-11b-vision-instruct', 'provider': 'meta', 'display_name': 'Llama 3.2 11B Vision Instruct', 'tokenizer': 'meta-llama/Llama-3.2-11B-Vision-Instruct', 'type': 'llm', 'capabilities': ['chat', 'vision'], 'server': 'text-generation-inference', 'parameters': {'max_input_tokens': 127999}}, {'name': 'codellama-13b-instruct-hf', 'provider': 'meta', 'display_name': 'CodeLlama 13B Instruct HF', 'tokenizer': 'meta-llama/CodeLlama-13b-Instruct-hf', 'type': 'llm', 'capabilities': ['chat'], 'server': 'text-generation-inference', 'parameters': {'max_input_tokens': 6144}}, {'name': 'codellama-13b-python-hf', 'provider': 'meta', 'display_name': 'CodeLlama 13B Python HF', 'tokenizer': 'meta-llama/CodeLlama-13b-Python-hf', '

#### Building Knowledge Base

In [3]:
# Importing dependencies
import requests
import os
from bs4 import BeautifulSoup
from newsapi import NewsApiClient
import re

# Creating directory to hold scraped articles
os.makedirs(name=f"../knowledge-bases/{keywords}_{source}")

# Creating newsapi client
newsapiclient = NewsApiClient(api_key="03122dc3b7b84ea29212ca965b40c7aa")

# Querying articles
articles = newsapiclient.get_everything(q=keywords, sources=source ,page_size=100)
articles = articles['articles']

# Scraping articles and saving in directory
for article in articles:
	url = article['url']
	response = requests.get(url=url)

	# Printing article content to directory if valid response
	if response.status_code==200:
		beautifulsoup = BeautifulSoup(response.content, "html.parser")
		article_paragraphs = beautifulsoup.find_all("p")

		# Cleaning article title
		article_title = re.sub(' ', '_', article['title'])

		with open(file=f"../knowledge-bases/{keywords}_{source}/{article_title}.txt", mode="w") as fp:
			for paragraph in article_paragraphs:
				paragraph_cleaned = str(paragraph.get_text()).strip()

				if paragraph_cleaned != "":
					fp.write(paragraph_cleaned)

#### Loading and Splitting Documents

In [4]:
# Importing dependencies
from langchain.document_loaders import DirectoryLoader
from langchain_text_splitters import CharacterTextSplitter

# Defining directory path
directory = f"../knowledge-bases/{keywords}_{source}"

# Creating tokenizer

# Creating loader and splitter
loader = DirectoryLoader(path=directory, glob="*.txt")
splitter = CharacterTextSplitter.from_tiktoken_encoder(encoding_name="cl100k_base", 
													   chunk_size=256, chunk_overlap=0)

# Loading and splitting documents
docs = loader.load_and_split(text_splitter=splitter)

Created a chunk of size 485, which is longer than the specified 256
Created a chunk of size 293, which is longer than the specified 256


#### Embedding and Storing Documents

In [5]:
# Importing dependencies
from langchain_dartmouth.embeddings import DartmouthEmbeddings
from langchain_core.vectorstores import InMemoryVectorStore

# Creating embeddings model
embeddings = DartmouthEmbeddings(model_name=embeddings_model_name, dartmouth_api_key=str(DARTMOUTH_API_KEY))

# Embedding documents and storing them in memory
vector_store = InMemoryVectorStore(embedding=embeddings)

for i in range(0, len(docs), 50):
	_ = vector_store.add_documents(docs[i: i+100])

#### Retrieval and Generation

In [8]:
# Importing dependencies
from langchain_dartmouth.llms import ChatDartmouthCloud
import json

# Initializing variable to hold output
output = ""

# Initializing variable referencing LLM
llm = ChatDartmouthCloud(model_name=llm_model_name)

# Open testing data file
with open(testing_data, 'r') as fp:
	test_data = json.load(fp)

counter = 1

# Iterating through each test data point
for tweet in test_data:

	# Retrieving most-similar documents
	query = tweet['Tweet']
	docs = vector_store.similarity_search(query, k=5)
	
	# Creating augmented prompt
	prompt = (
		"Classify this tweet as 'Real News' or 'Fake News': "
		+ query
		+ f"\n\nConsider the following info: \n\n"
	)

	for doc in docs:
		prompt += doc.page_content + "\n--\n"
	
	prompt = prompt + "Only respond with the classification."

	# Querying LLM and printing response to file
	response = llm.invoke(prompt)
	output = response.pretty_repr() + output

ValueError: {'detail': 'You\'ve exceeded the daily usage limit (1000 credits) for the paid AI models.\n                    IMPORTANT: Click the "New Chat" button and select one of the free models (ex. Llama 3.1) to start a new chat session.\n                    '}

#### Evaluation 

In [ ]:
# Importing dependencies
import json
from sklearn.metrics import precision_score, recall_score, accuracy_score, confusion_matrix, f1_score
import io

# Loading true labels
with open(testing_data) as fp_input:
	input_data = json.load(fp_input)

	# Encoding true labels as 0s and 1s
	y_actual = []

	for entry in input_data:
		label = entry['Label']

		if label == "Real News":
			y_actual.append(1)
		elif label == "Fake News":
			y_actual.append(0)

# Extracting predicted labels
y_pred = []

# Encoding true labels as 0s and 1s
with io.StringIO(output) as fp_output:

	for line in fp_output:
		if "Real News" in line:
			y_pred.append(1)
		elif "Fake News" in line:
			y_pred.append(0)

# Calculating and printing metrics
recall = recall_score(y_true=y_actual, y_pred=y_pred)
precision = precision_score(y_true=y_actual, y_pred=y_pred)
accuracy = accuracy_score(y_true=y_actual, y_pred=y_pred)
f1score = f1_score(y_true=y_actual, y_pred=y_pred)
confusion_matrix = confusion_matrix(y_true=y_actual, y_pred=y_pred)

print("Recall: ", recall)
print("Precision: ",precision)
print("Accuracy: ",accuracy)
print("F1 Score", f1score)
print("Confusion matrix: ",confusion_matrix)
		
		

### **GPT-4o-Mini w/o Augmentation** ###

#### Generation

In [ ]:
# Importing dependencies
from langchain_dartmouth.llms import ChatDartmouthCloud
import json

# Initializing variable to hold output
output = ""

# Initializing variable referencing LLM
llm = ChatDartmouthCloud(model_name=llm_model_name)

# Open testing data file
with open(testing_data, 'r') as fp:
	test_data = json.load(fp)

# Iterating through each test data point
for tweet in test_data:

	# Creating non-augmented prompt
	query = tweet['Tweet']
	prompt = (
		"Classify this tweet as 'Real News' or 'Fake News': "
		+ query
		+ "Only respond with the classification."
	)

	# Querying LLM and printing response to file
	response = llm.invoke(prompt)
	output = response.pretty_repr() + output

#### Evaluation

In [ ]:
# Importing dependencies
import json
from sklearn.metrics import precision_score, recall_score, accuracy_score, confusion_matrix, f1_score
import io

# Loading true labels
with open(testing_data) as fp_input:
	input_data = json.load(fp_input)

	# Encoding true labels as 0s and 1s
	y_actual = []

	for entry in input_data:
		label = entry['Label']

		if label == "Real News":
			y_actual.append(1)
		elif label == "Fake News":
			y_actual.append(0)

# Extracting predicted labels
y_pred = []

# Encoding true labels as 0s and 1s
with io.StringIO(output) as fp_output:

	for line in fp_output:
		if "Real News" in line:
			y_pred.append(1)
		elif "Fake News" in line:
			y_pred.append(0)


print(len(y_actual))
print(len(y_pred))




# Calculating and printing metrics
recall = recall_score(y_true=y_actual, y_pred=y_pred)
precision = precision_score(y_true=y_actual, y_pred=y_pred)
accuracy = accuracy_score(y_true=y_actual, y_pred=y_pred)
f1score = f1_score(y_true=y_actual, y_pred=y_pred)
confusion_matrix = confusion_matrix(y_true=y_actual, y_pred=y_pred)

print("Recall: ", recall)
print("Precision: ",precision)
print("Accuracy: ",accuracy)
print("F1 Score", f1score)
print("Confusion matrix: ",confusion_matrix)